In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [9]:
ALL_CSV_PATH = [
    "/workspace/plane-nerf/outputs/jackal_floor_training_data_1/plane-nerf-fast/2024-03-18_221148/eval_results_2024-03-18_225408.csv",
    "/workspace/plane-nerf/outputs/jackal_floor_training_data_1/plane-nerf-fast/2024-03-18_221148/eval_results_2024-03-20_150641.csv",
    "/workspace/plane-nerf/outputs/jackal_floor_training_data_1/plane-nerf-fast/2024-03-18_221148/eval_results_2024-03-20_104355.csv"
]

In [10]:
all_translation = []
all_rotation = []

for CSV_PATH in ALL_CSV_PATH:
    #Open CSV file as 2d numpy array
    csv_data = np.genfromtxt(CSV_PATH, delimiter=',')
    #If nan values are present, replace them with 0
    csv_data = np.nan_to_num(csv_data)
    iteration = csv_data[:, 0::3]
    translation = csv_data[:, 1::3]
    rotation = csv_data[:, 2::3]
    all_translation.append(translation)
    all_rotation.append(rotation)

all_translation = np.array(all_translation)
all_rotation = np.array(all_rotation)

Final translation error: 0.074378232418481 m
Final rotation error: 4.899139415102579 degrees
Standard deviation of final translation error: 0.057212584666006046 m
Standard deviation of final rotation error: 4.911244390111997 degrees
[0.425 0.155 0.335 0.07  0.01  0.005] [0.   0.05 0.1  0.15 0.2  0.25]
[0.51, 0.295, 0.19, 0.005, 0.0, 0.0] [0, 5, 10, 15, 20, 25]
Final translation error: 0.19135102925652717 m
Final rotation error: 19.515529025738026 degrees
Standard deviation of final translation error: 0.06649475642466694 m
Standard deviation of final rotation error: 3.465886779514715 degrees
[0.04  0.02  0.17  0.345 0.275 0.075] [0.   0.05 0.1  0.15 0.2  0.25]
[0.0, 0.01, 0.08, 0.425, 0.455, 0.03] [0, 5, 10, 15, 20, 25]
Final translation error: 0.2847199140678085 m
Final rotation error: 29.868355166456222 degrees
Standard deviation of final translation error: 0.07507536511167023 m
Standard deviation of final rotation error: 2.9262055112025607 degrees
[0.015 0.01  0.025 0.05  0.17  0.295

In [13]:
#Plot 3 graphs for PSNR, SSIM, LPIPS side by side
fig, axs = plt.subplots(1, 2, figsize=(20, 5))

#Increase font size
plt.rcParams.update({'font.size': 12})

#Increase font size of axis
for ax in axs:
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(14) 

graph = 0
for translation, rotation in zip(all_translation, all_rotation):
    final_translation_error = translation[:,-1]
    final_rotation_error = rotation[:,-1]
    print(f"Final translation error: {final_translation_error.mean()} m")
    print(f"Final rotation error: {final_rotation_error.mean() * 180 / math.pi} degrees")

    #Find standard deviation of final translation and rotation error
    print(f"Standard deviation of final translation error: {final_translation_error.std()} m")
    print(f"Standard deviation of final rotation error: {final_rotation_error.std() * 180 / math.pi} degrees")

    #Plot histogram of final translation and rotation error

    bin = []
    percentage = []
    cnt = 6
    for x in range(cnt):
        i = x * 0.05
        bin.append(i)
        percentage.append(len(final_translation_error[(final_translation_error >= i) & (final_translation_error < i+0.05)])/200)

    bin = np.array(bin)
    percentage = np.array(percentage)

    print(percentage,bin)

    bar_width = np.array([0.05] * cnt) * 0.27

    #Draw bar chart side by side
    axs[0].bar(bin+bar_width*graph, percentage, width=bar_width, align="edge")
    axs[0].set_xlabel("Error (degrees)")
    axs[0].set_ylabel("Percentage")
    axs[0].set_ylim(0, 1)

    bin = []
    percentage = []
    cnt = 6
    for x in range(cnt):
        i = x * 5
        bin.append(i)
        percentage.append(len(final_rotation_error[(final_rotation_error/math.pi *180 >= i) & (final_rotation_error/math.pi*180 < i+5)])/200)

    print(percentage,bin)

    bar_width = np.array([5] * cnt) * 0.27

    axs[1].bar(bin+bar_width*graph, percentage, width=bar_width, align="edge")
    axs[1].set_xlabel("Error (degrees)")
    axs[1].set_ylabel("Percentage")
    axs[1].set_ylim(0, 1)

    graph+=1

axs[0].legend(["10 cm/ 10 degrees", "20 cm/ 20 degrees", "30 cm/ 30 degrees"])
axs[1].legend(["10 cm/ 10 degrees", "20 cm/ 20 degrees", "30 cm/ 30 degrees"])

In [12]:
#Plot 3 graphs for PSNR, SSIM, LPIPS side by side
fig, axs = plt.subplots(1, 2, figsize=(20, 5))

#Increase font size
plt.rcParams.update({'font.size': 12})

#Increase font size of axis
for ax in axs:
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(14)

for translation, rotation in zip(all_translation, all_rotation):
    #Draw ratio that falls below 5 cm 5 degrees against iteration
    axs[0].plot((translation < 0.05).mean(axis=0))
    axs[0].set_ylabel("% of Translation error < 5cm")
    axs[0].set_xlabel("Number of Iteration")
    axs[0].set_ylim(0, 1)
    axs[0].set_xlim(0,1000)

    axs[1].plot(((rotation < 0.05).mean(axis=0)))
    axs[1].set_ylabel("% of Rotation error < 5 degrees")
    axs[1].set_xlabel("Number of Iteration")
    axs[1].set_ylim(0, 1)
    axs[1].set_xlim(0,1000)
axs[0].legend(["10 cm/ 10 degrees", "20 cm/ 20 degrees", "30 cm/ 30 degrees"])
axs[1].legend(["10 cm/ 10 degrees", "20 cm/ 20 degrees", "30 cm/ 30 degrees"])